In [1]:
import numpy as np
import pandas as pd
from app2 import *

['0ngwQpVCEUhKingLoOpd1I', '7zGVMRHuPm0JudlT5kgWog', '2o081YaHjPiDIgOtIyw8Hp', '1qbUsgOaOMDB7UYNQKFBSZ', '7BP5onQdR6vqqa78qT9AgV', '2NebeZvLy2MsWKiAFk6R7o']
0ngwQpVCEUhKingLoOpd1I
46
7zGVMRHuPm0JudlT5kgWog
35
2o081YaHjPiDIgOtIyw8Hp
1qbUsgOaOMDB7UYNQKFBSZ
7BP5onQdR6vqqa78qT9AgV
100
2NebeZvLy2MsWKiAFk6R7o
89


In [2]:
from __future__ import print_function

In [3]:
# preparing my dataframe
# Each row is an example, We are attempting to predict the playlist name, the other columns are features
for i in range(len(playlist_namelist)):
    tracklist = pd.read_json(playlist_namelist[i])
    tracklist = tracklist.set_index("id")
    tracklist = tracklist.drop(["analysis_url","track_href","uri","type"],1)
    tracklist["playlist"] = playlist_namelist[i][:-5]
    if i == 0:
        all_tracks = tracklist
    else:
        all_tracks = all_tracks.append(tracklist)
for i in range(len(track_stuff_namelist)):
    track_stuff = pd.read_json(track_stuff_namelist[i])
    track_stuff = track_stuff.T
    track_stuff = track_stuff.set_index("id")
    if i == 0:
        all_tracks_2 = track_stuff
    else:
        all_tracks_2 = all_tracks_2.append(track_stuff)
all_tracks["artist"] = all_tracks_2["artist"]
all_tracks["popularity"] = all_tracks_2["popularity"]
all_tracks["popularity"] = pd.to_numeric(all_tracks["popularity"])
all_tracks.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,playlist,artist,popularity
id,,,,,,,,,,,,,,,,
7ltsfuHdqTZ5LwPpDy1q0v,0.1860,0.786,416613,0.579,0.000000,2,0.1180,-14.913,1,0.1420,90.012,4,0.481,Hindi,Sonu Nigam,45
0OXQa6GOzxeYWFopEmASw9,0.0493,0.805,425413,0.670,0.000050,2,0.2030,-13.187,1,0.0601,122.992,4,0.916,Hindi,A.R. Rahman,18
6B58TJqjGLQuuZBhQ3UnEA,0.1200,0.767,309107,0.503,0.000096,9,0.1580,-13.424,1,0.0828,141.996,4,0.483,Hindi,Lata Mangeshkar,35
0IYpKZhCeoXB85BslBg3q5,0.4190,0.646,327227,0.908,0.000002,11,0.3810,-4.680,0,0.0988,104.238,4,0.881,Hindi,Jatin-Lalit,45
7CcAKQatXMmlr7wi6BaQsI,0.0815,0.493,297927,0.947,0.009370,1,0.0552,-4.188,0,0.2060,127.986,4,0.568,Hindi,Sanjay Leela Bhansali,40


In [4]:
msk = np.random.randn(len(all_tracks)) < .8
training = all_tracks[msk]
testing= all_tracks[~msk]

In [5]:
def list_of_lister(df):
    row_list = []
    for index, row in df.iterrows():
        row_list.append(list(row))
    return row_list
all_tracks_input = list_of_lister(all_tracks)
training_data = list_of_lister(training)
testing_data = list_of_lister(testing)

In [6]:
# column labels
column_labels = list(all_tracks.columns)
print(column_labels)

['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'playlist', 'artist', 'popularity']


In [7]:
# Finding the number of unique playlists within the dataset
def unique_vals(rows, col):
    return set(row[col] for row in rows)
unique_vals(all_tracks_input, 13)

{'Frictionless', 'Hindi', "I'm a god", 'Peak Trap'}

In [8]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[13]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [9]:
class_counts(all_tracks_input)

{'Frictionless': 100, 'Hindi': 46, "I'm a god": 89, 'Peak Trap': 35}

In [10]:
def  is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [11]:
class  Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            column_labels[self.column], condition, str(self.value))

In [12]:
q = Question(13, 'Frictionless')

In [13]:
example = all_tracks.iloc[1]

In [14]:
q.match(example)

False

In [15]:
# This function partitions a dataset and checks if it matches the question
# If it matches the question it is added to true_rows
# Else it is added to false_rows
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [16]:
#######
# Demo:
# Let's partition the training data based on whether or not the song is in the playlist Frictionless.
true_rows, false_rows = partition(all_tracks_input, Question(13, "Frictionless"))
# This will contain all the 'Red' rows.
true_rows

[[0.922,
  0.422,
  269560,
  0.264,
  0.0,
  8,
  0.132,
  -7.064,
  1,
  0.0322,
  119.93,
  4,
  0.331,
  'Frictionless',
  'John Legend',
  83],
 [0.09720000000000001,
  0.6990000000000001,
  252013,
  0.601,
  1.17e-05,
  0,
  0.562,
  -6.48,
  1,
  0.0304,
  98.04,
  4,
  0.434,
  'Frictionless',
  'Childish Gambino',
  74],
 [0.15,
  0.516,
  210680,
  0.718,
  0.016300000000000002,
  5,
  0.167,
  -8.571,
  0,
  0.257,
  129.857,
  4,
  0.588,
  'Frictionless',
  'Childish Gambino',
  61],
 [0.41000000000000003,
  0.716,
  200747,
  0.35000000000000003,
  0.00134,
  0,
  0.0927,
  -11.094,
  1,
  0.053700000000000005,
  129.842,
  4,
  0.178,
  'Frictionless',
  'Frank Ocean',
  71],
 [0.149,
  0.47300000000000003,
  234213,
  0.461,
  0.0,
  6,
  0.09730000000000001,
  -6.474,
  0,
  0.333,
  165.985,
  4,
  0.615,
  'Frictionless',
  'Childish Gambino',
  72],
 [0.871,
  0.289,
  223600,
  0.254,
  0.0745,
  7,
  0.135,
  -11.218,
  0,
  0.047400000000000005,
  77.499,
  4,
 

In [17]:
false_rows

[[0.186,
  0.786,
  416613,
  0.579,
  0.0,
  2,
  0.11800000000000001,
  -14.913,
  1,
  0.14200000000000002,
  90.012,
  4,
  0.481,
  'Hindi',
  'Sonu Nigam',
  45],
 [0.049300000000000004,
  0.805,
  425413,
  0.67,
  5e-05,
  2,
  0.203,
  -13.187,
  1,
  0.0601,
  122.992,
  4,
  0.916,
  'Hindi',
  'A.R. Rahman',
  18],
 [0.12,
  0.767,
  309107,
  0.503,
  9.56e-05,
  9,
  0.158,
  -13.424,
  1,
  0.0828,
  141.996,
  4,
  0.483,
  'Hindi',
  'Lata Mangeshkar',
  35],
 [0.419,
  0.646,
  327227,
  0.908,
  1.57e-06,
  11,
  0.381,
  -4.68,
  0,
  0.0988,
  104.238,
  4,
  0.881,
  'Hindi',
  'Jatin-Lalit',
  45],
 [0.0815,
  0.493,
  297927,
  0.9470000000000001,
  0.009370000000000002,
  1,
  0.055200000000000006,
  -4.188,
  0,
  0.20600000000000002,
  127.986,
  4,
  0.5680000000000001,
  'Hindi',
  'Sanjay Leela Bhansali',
  40],
 [0.177,
  0.34900000000000003,
  368547,
  0.596,
  5.29e-06,
  2,
  0.336,
  -9.313,
  1,
  0.0364,
  120.009,
  4,
  0.329,
  'Hindi',
  'Visha

In [18]:
# Calculating gini coefficient of rows
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [19]:
gini(true_rows)

0.0

In [20]:
gini(false_rows)

0.6103114186851211

In [21]:
gini(all_tracks_input)

0.7083401920438956

In [22]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [23]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.7041471212863778

In [24]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(-1, 50))
info_gain(true_rows, false_rows, current_uncertainty)

0.11686511217273124

In [25]:
true_rows

[[0.45,
  0.59,
  408133,
  0.875,
  1.18e-05,
  5,
  0.164,
  -5.294,
  1,
  0.171,
  96.915,
  4,
  0.736,
  'Hindi',
  'Jatin-Lalit',
  50],
 [0.334,
  0.532,
  321240,
  0.529,
  0.000144,
  0,
  0.078,
  -8.402,
  1,
  0.0352,
  81.989,
  4,
  0.295,
  'Hindi',
  'Shankar-Ehsaan-Loy',
  51],
 [0.0273,
  0.599,
  197587,
  0.888,
  0.0,
  11,
  0.28,
  -2.931,
  0,
  0.0514,
  121.05,
  4,
  0.833,
  'Peak Trap',
  'Big Time Rush',
  54],
 [0.0056,
  0.531,
  179493,
  0.766,
  0.0,
  8,
  0.201,
  -7.692,
  1,
  0.0582,
  130.048,
  4,
  0.532,
  'Peak Trap',
  'Original Soundtrack',
  51],
 [0.00124,
  0.454,
  202067,
  0.708,
  0.00018099999999999998,
  10,
  0.0679,
  -4.526,
  0,
  0.0675,
  192.18,
  4,
  0.483,
  'Peak Trap',
  'Miley Cyrus',
  73],
 [0.7020000000000001,
  0.443,
  195147,
  0.47200000000000003,
  0.0,
  5,
  0.07980000000000001,
  -14.567,
  1,
  0.258,
  62.402,
  4,
  0.847,
  'Peak Trap',
  'Alan Menken',
  66],
 [1.2800000000000001e-05,
  0.527,
  1740

In [26]:
false_rows

[[0.186,
  0.786,
  416613,
  0.579,
  0.0,
  2,
  0.11800000000000001,
  -14.913,
  1,
  0.14200000000000002,
  90.012,
  4,
  0.481,
  'Hindi',
  'Sonu Nigam',
  45],
 [0.049300000000000004,
  0.805,
  425413,
  0.67,
  5e-05,
  2,
  0.203,
  -13.187,
  1,
  0.0601,
  122.992,
  4,
  0.916,
  'Hindi',
  'A.R. Rahman',
  18],
 [0.12,
  0.767,
  309107,
  0.503,
  9.56e-05,
  9,
  0.158,
  -13.424,
  1,
  0.0828,
  141.996,
  4,
  0.483,
  'Hindi',
  'Lata Mangeshkar',
  35],
 [0.419,
  0.646,
  327227,
  0.908,
  1.57e-06,
  11,
  0.381,
  -4.68,
  0,
  0.0988,
  104.238,
  4,
  0.881,
  'Hindi',
  'Jatin-Lalit',
  45],
 [0.0815,
  0.493,
  297927,
  0.9470000000000001,
  0.009370000000000002,
  1,
  0.055200000000000006,
  -4.188,
  0,
  0.20600000000000002,
  127.986,
  4,
  0.5680000000000001,
  'Hindi',
  'Sanjay Leela Bhansali',
  40],
 [0.177,
  0.34900000000000003,
  368547,
  0.596,
  5.29e-06,
  2,
  0.336,
  -9.313,
  1,
  0.0364,
  120.009,
  4,
  0.329,
  'Hindi',
  'Visha

In [27]:
def  find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    index_features = [0,1,3,7,9,15]  # index of the features in each list

    for num in index_features:  # for each feature

        values = set([row[num] for row in rows])  # unique values for each feature

        for val in values:  # for each value

            question = Question(num, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [28]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is popularity >= 52?

In [29]:
# This class is used to represent leaf nodes in our decision tree, leaf nodes classify data. The class Leaf holds
# a dictionary of target values and the number of times it appears in the rows that have made it to this leaf node
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [30]:
# Next we will initialize a class called Decision_Node, in decision trees, a decision node asks a question, this class holds
# reference to a question and to the child_nodes
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [31]:
# Now we have a function called build_tree, which takes in the data as an input and builds a tree recursively
def build_tree(rows):
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [32]:
# Next we have a function called print_tree which basically allows us to print our tree in a pretty format
def print_tree(node, spacing = ""):
    if isinstance(node, Leaf):
        print(spacing + "Predict:", node.predictions)
        return
    print(spacing + str(node.question))
    
    print(spacing + "--> True")
    print_tree(node.true_branch, spacing + "    ")
    
    print(spacing + "--> False")
    print_tree(node.false_branch, spacing + "    ")
    

In [33]:
my_tree = build_tree(training_data)

In [34]:
print_tree(my_tree)

Is popularity >= 52?
--> True
    Is acousticness >= 0.0786?
    --> True
        Is danceability >= 0.8160000000000001?
        --> True
            Is speechiness >= 0.124?
            --> True
                Is acousticness >= 0.517?
                --> True
                    Is popularity >= 63?
                    --> True
                        Predict: {"I'm a god": 1}
                    --> False
                        Predict: {'Frictionless': 1}
                --> False
                    Predict: {"I'm a god": 9}
            --> False
                Is popularity >= 65?
                --> True
                    Is speechiness >= 0.12?
                    --> True
                        Predict: {'Frictionless': 1}
                    --> False
                        Predict: {"I'm a god": 2}
                --> False
                    Predict: {'Frictionless': 3}
        --> False
            Is loudness >= -4.408?
            --> True
                Is spee

In [35]:
# This next function is called classify, which once again is a recursive function, it takes in a given row of data in
# as an input and node, which is a decision node output as another input
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [36]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{'Hindi': 19}

In [37]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [38]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{'Hindi': '100%'}

In [39]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[96], my_tree))
#######

{'Frictionless': '100%'}

In [40]:
row_list = []
item_list = []
from sklearn.metrics import accuracy_score
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[13], print_leaf(classify(row, my_tree))))
    row_list.append(row[13])
    for item in print_leaf(classify(row,my_tree)).keys():
        item_list.append(item)
print(accuracy_score(row_list, item_list))

Actual: Hindi. Predicted: {'Peak Trap': '100%'}
Actual: Hindi. Predicted: {'Peak Trap': '100%'}
Actual: Hindi. Predicted: {'Peak Trap': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Hindi. Predicted: {'Peak Trap': '100%'}
Actual: Hindi. Predicted: {'Peak Trap': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Hindi. Predicted: {'Peak Trap': '100%'}
Actual: Hindi. Predicted: {'Hindi': '100%'}
Actual: Peak Trap. Predicted: {'Hindi': '100%'}
Actual: Peak Trap. Predicted: {'Hindi': '100%'}
Actual: Peak Trap. Predicted: {'Peak Trap': '100%'}
Actual: Peak Trap. Predicted: {"I'm a god": '100%'}
Actual: Peak Trap. Predicted: {"I'm a god": '100%'}
Actual: Peak Trap. Predicted: {'Peak Trap': '100%'}
Actual: Peak Trap. Predicted: {'Frictionless': '100%'}
Actual: Peak Trap. Predicted: {'Frictionless': '100%'}
Actual: Peak Trap. Predicted: {'Hindi': '100%'}
Actual: Peak Trap. Predicted: {'Peak Trap': '100%'}
Actual: Frictionless. 

In [41]:
len(testing_data)

63

0.515625